In [ ]:
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.stem.snowball import SnowballStemmer

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

spark = sparknlp.start()

In [ ]:
test_dataset_df = pd.read_pickle("amazon_clean_data.pkl")

In [ ]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document") 

# Can take in document or sentence columns
t5 = T5Transformer.pretrained(name='t5_base',lang='en')\
    .setInputCols('document')\
    .setOutputCol("T5")

In [ ]:
# Set the task on T5
t5.setTask('cola sentence:')

# Build pipeline with T5
pipe_components = [documentAssembler,t5]
pipeline = Pipeline().setStages( pipe_components)

In [ ]:
# Get some test sentences
sentences = test_dataset_df['question'].sample(n=20).tolist()
# Lets just use short sentences for now
sentences = [[x] for x in sentences if len(x) < 90]
df = spark.createDataFrame(sentences).toDF("text")

#Predict on text data with T5
model = pipeline.fit(df)
annotated_df = model.transform(df)
annotated_df.select(['text','t5.result']).show(truncate=False)

### In pandas DF format

In [ ]:
annotated_df.select(['text','t5.result']).toPandas()